<a href="https://colab.research.google.com/github/ojcharles/notebooks/blob/main/oscars_paper_clustering_by_NLP_herpes_resistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scientific Article Literature Clustering by NLP

Oscar Charles 2021

This is a notebook that clusters journals, and can help you find similar papers to those you already know are important.

What you need:


*   A pubmed query. Example included
*   A list of DOI's for papers you already care about. Example included



Question: Can we use Natural Language processing to take a set of abstracts (strings) -> tokens -> vectorisation -> clustering of abstracts by similarity? We have a set of papers we want to cluster close together, can we get them too such that we can find similar papers we may have missed?



In [ ]:
# install all the python stuff I need
# get the requirements file
!wget https://raw.githubusercontent.com/ojcharles/snippets/main/nlp_lit_clustering_requirements.txt

# install the packages
!pip install -r nlp_lit_clustering_requirements.txt

# install the spacy model
# A full spaCy pipeline for biomedical data with a larger vocabulary and 50k word vectors.
! python -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz

--2022-06-11 10:56:16--  https://raw.githubusercontent.com/ojcharles/snippets/main/nlp_lit_clustering_requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 561 [text/plain]
Saving to: ‘nlp_lit_clustering_requirements.txt.1’

nlp_lit_clustering_ 100%[===================>]     561  --.-KB/s    in 0s      

2022-06-11 10:56:16 (23.9 MB/s) - ‘nlp_lit_clustering_requirements.txt.1’ saved [561/561]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl (13.9 MB)
  Using cached matplotlib-3.5.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached oauthl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120.2 MB 46 kB/s 
     |████████████████████████████████| 6.0 MB 5.2 MB/s 
  Created wheel for en-core-sci-md: filename=en_core_sci_md-0.5.0-py3-none-any.whl size=120252807 sha256=e773f96a1a5bdc5bfa30ebbc1fd2a72f50fadb7d011dca625b77b28a32ea0272
  Stored in directory: /root/.cache/pip/wheels/c2/9a/de/7c6ba2bbf7d24e6f54810fd43b09b30a813dcaaabf33c0f941
Successfully built en-core-sci-md
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.1
    Uninstalling spacy-3.2.1:
      Successfully uninstalled spacy-3.2.1


In [ ]:
# also install EDirect for pubmed APi calls
!  yes | sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



Getting paper data from Pubmed and parseing the json

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xml.etree.ElementTree as ET
import bokeh

In [ ]:
# pubmed has all papers, epmc only those that are openly accessible.

# Conduct a PubMed search and retrieve the results as a list of PMIDs
! ${HOME}/edirect/esearch -db pubmed -query "((resistance)) AND ((ribavirin) OR (favipiravir) OR (remdesivir) OR (EIDD-2801) OR (molnupiravir)) "   | ${HOME}/edirect/efetch -format uid > pubmed.txt

# how many entries?
! wc -l pubmed.txt 

# take those identified pubmed ids of interest and 
! ${HOME}/edirect/efetch -db pubmed -input pubmed.txt -format xml > pubmed.xml

# sanity check
! head -5 pubmed.xml

1821 pubmed.txt
<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE PubmedArticleSet>
<PubmedArticleSet>
  <PubmedArticle>
    <MedlineCitation Status="MEDLINE" IndexingMethod="Curated" Owner="NLM">


In [ ]:
! sed -n 54814p pubmed.xml

        <ArticleTitle>Most Promising Compounds for Treating COVID-19 and Recent Trends in Antimicrobial & Antifungal Agents.</ArticleTitle>


In [ ]:
# parse xml
dict_ = {'doi': [], 'title': [], 'abstract':[]}

tree = ET.parse('pubmed.xml') # download from a search in epmc
root = tree.getroot()

search_doi = 'PubmedData/ArticleIdList/ArticleId[@IdType="doi"]'
search_title = 'MedlineCitation/Article/ArticleTitle'
search_abstract = 'MedlineCitation/Article/Abstract/AbstractText'

for entry in root.iter('PubmedArticle'):
    if entry.find(search_doi) is None:
        continue
    else:
        doi = entry.find(search_doi).text
        
    if entry.find(search_title) is None:
        continue
    else:
        title = entry.find(search_title).text
        
    if entry.find(search_abstract) is None:
        continue
    else:
        abstractText = entry.find(search_abstract).text
    #print(doi, title)
    dict_['doi'].append(doi)
    dict_['title'].append(title)
    dict_['abstract'].append(abstractText)


ParseError: ignored

In [ ]:
# format the dict to a table
df = pd.DataFrame(dict_, columns=['doi', 'title','abstract'])
df.head()

In [ ]:
# how many papers are left with al the required info?
len(df)

## Paper Encoding

Here we take the text and identify each token "word" and use the spacy model to find the relationships etc.

In [ ]:
#NLP 
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
# spacy.prefer_gpu() # if you have a GPU do this its so much faster

# using scispacy we can add support for scientific abbreviations
from scispacy.abbreviation import AbbreviationDetector

parser = spacy.load("en_core_sci_md") # scispacy model
# Add the abbreviation pipe to the spacy pipeline.
parser.add_pipe("abbreviation_detector")


In [ ]:
import string

punctuations = string.punctuation
stopwords = list(STOP_WORDS)
stopwords[:10]

In [ ]:
# stop words are words that are removed.
# as We don't want a cluster per virus, per drug etc but generally paper with novel rna resmuts we add these extra stopwords.
custom_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 
    'al.', 'Elsevier', 'PMC', 'CZI', 'www',
    "Hepatitis C", "Herpes", "Simplex 1", "Simplex 2", "Cytomegalovirus", "Coronavirus", "HCMV", "CMV", "HSV1", "HSV2", 
    "influenza A", "(H1N1)", "HIV-1", "HIV", "Hepatitis B", "HCV", "HBV",
    "SARS-COV-2", "COVID-19", "COVID", "human immunodeficiency virus", "HIV-RNA", "subtype", "genotype", "subtype",
    "avian influenza", "syncytial", "polio", "poliovirus", "chikungunya"
    
]

for w in custom_stop_words:
    if w not in stopwords:
        stopwords.append(w)

In [ ]:
# Parser

parser.max_length = 7000000

def call_tokenizer(df):
    df["processed_abstract"] = df["abstract"].apply(spacy_tokenizer)
    return df

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [ ]:
from multiprocessing import  Pool

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
%%capture --no-stdout
# scispacy abbreviation complains each time a abstract has no abbreviations it knows.... stop that warning.
# tokenise
# single core - single gpu
df["processed_abstract"] = df["abstract"].apply(spacy_tokenizer)
# many cpu
#%time df = parallelize_dataframe(df=df, func=call_tokenizer, n_cores=16)

### save


In [ ]:
import pickle
pickle.dump(df, open("tokenised_df.p", "wb" ))

## paper vectoriser


In [ ]:
df = pickle.load(open("tokenised_df.p", "rb"))
#df = df.sample(600000)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def vectorize(text, maxx_features):
    
    vectorizer = TfidfVectorizer(max_features=maxx_features)
    X = vectorizer.fit_transform(text)
    return X

In [ ]:
text = df['processed_abstract'].values
%time X = vectorize(text, 2 ** 12)
X.shape

##Cluster

In [ ]:
# as well as x and y space, lets colour each paper by Kmeans clustering
from sklearn.cluster import KMeans
k = 20
kmeans = KMeans(n_clusters=k, random_state=42)
%time y_pred = kmeans.fit_predict(X)
df['y'] = y_pred

## Plotting

In [ ]:
# reduce dimensions
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

pca = PCA(n_components=30, random_state=42)
%time X_embedded_pca= pca.fit_transform(X.toarray())
X_embedded_pca.shape

In [ ]:
# t-sne
from sklearn.manifold import TSNE
X_embedded_tsne = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(X_embedded_pca)
df['x_tsne'] = X_embedded_tsne[:,0] 
df['y_tsne'] = X_embedded_tsne[:,1]

#from tsnecuda import TSNE
#%time X_embedded_tsne = TSNE(n_components=2, learning_rate=30, n_iter=500000, verbose=1).fit_transform(X_embedded_pca)

In [ ]:
# identify which papers we know are of interest, then make them squares. maybe we need to hide the virus name from the model?
key_doi=["10.1371/journal.ppat.1009929",
"10.1128/mBio.00221-18",
"10.1128/JVI.01965-17",
"10.1073/pnas.1811345115",
"10.1093/jac/dku209",
"10.1128/AAC.01073-16",
"10.1073/pnas.1232294100",
"10.1128/JVI.02139-12",
"10.1016/j.antiviral.2013.07.008",
"10.1371/journal.ppat.1001163",
"10.1128/JVI.00289-14",
"10.1073/pnas.1111650108",
"10.1128/JVI.79.4.2346-2355.2005",
"10.1128/JVI.03594-13",
"10.1128/JVI.01297-08",
"10.1371/journal.ppat.1003877",
"10.1128/JVI.01528-14",
        "10.1128/JVI.00367-19",
        "10.1128/JVI.00078-16",
        "10.1016/j.jmii.2017.03.004",
        "10.1038/ncomms5794",
        "10.1074/jbc.C112.401471",
        "10.1099/jgv.0.000316",
        "10.3791/2953",
        "10.1099/jgv.0.000682",
        "10.1371/journal.ppat.1001163",
        "10.1016/j.bbrc.2013.12.071",
        "10.1038/nm1726",
        "10.1371/journal.ppat.1005010"]
df['key'] = df['doi'].isin(key_doi)
df[df['doi'].isin(key_doi)]

In [ ]:
# we want bokeh to treat y (cluster) as discrete not continuous 
df['y'] = df['y'].apply(str)
df.dtypes

In [ ]:
# make a pretty plot
from bokeh.plotting import ColumnDataSource, figure, output_notebook, show, output_file, save
from bokeh.palettes import d3
import bokeh.models as bmo
#output_file("toolbar.html")
output_notebook()

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("doi", "@doi"),
    ("title", "@title"),
]

# use whatever palette you want...
palette = d3['Category20'][len(df['y'].unique())]
color_map = bmo.CategoricalColorMapper(factors=df['y'].unique(),
                                   palette=palette)

p = figure(width=800, height=800, tooltips=TOOLTIPS,
           title="Mouse over the dots")
p.scatter('x_tsne', 'y_tsne', size=5, source=df[df['key'] == False], color={'field': 'y', 'transform': color_map})
p.scatter('x_tsne', 'y_tsne', size=8, source=df[df['key'] == True], color="black", marker="square")
show(p)
